In [1]:
import requests
import json
import pandas as pd
from TokenManager import TokenManager
from InContactReport import InContactReport
import base64
import csv
import datetime
from io import StringIO 

In [3]:
# API access token
token = TokenManager('creds.yml')

# ID of the report to run
report_id = '524'

In [4]:
# Base URL
url = 'https://api-c30.incontact.com/inContactAPI/services/v17.0/report-jobs/datadownload/{}'.format(report_id)

# Request Headers
headers = {'Authorization': 'Bearer {}'.format(token()),
           'Content-Type': 'x-www-form-urlencoded',
           'Accept': 'application/json'
           }


params = {'saveAsFile': 'false',
          'includeHeaders': 'true',
          'startDate': '2020-01-01T00:00:00.000',
          'endDate': '2020-01-02T00:00:00.000'
          }

In [5]:
response = requests.post(url, params=params, headers=headers)
response.raise_for_status()

print(response.status_code == 200)

response_raw = response.json()['file']

data_string = StringIO(base64.b64decode(response_raw).decode('ascii'))

df = pd.read_csv(data_string, sep=',')

print(df.shape)
df.head()

True
(2048, 28)


,contact_id,master_contact_id,Contact_Code,media_name,contact_name,ANI_DIALNUM,skill_no,skill_name,campaign_no,campaign_name,agent_no,agent_name,team_no,team_name,disposition_code,SLA,start_date,start_time,PreQueue,InQueue,Agent_Working_Time,PostQueue,Total_Time,Abandon_Time,Routing_Time,abandon,callback_time,Logged
0,123178879053,123178879053,18959143,Phone Call,18006802345,+13233601383,4149350,NA - ENG - Music Services - IB,2523910,Americas - Eng Care,7244739,Maria Gonzalez,2422583,NA - ENG - CR - Tier 1,0,0,01/01/2020,00:00:12,18,27,1368,0,1413,0,27,N,0,Y
1,123178879430,123178879430,18959143,Phone Call,18006802345,+13343992356,4069210,Default Inbound - NO AGENT,2522967,Admin,0,NaN,0,NaN,0,-1,01/01/2020,00:00:37,37,0,0,0,37,0,0,N,0,N
2,123178879551,123178879551,18959143,Phone Call,18006802345,+12516231110,4149348,NA - ENG - Care - IB,2523910,Americas - Eng Care,7244730,?jose chinchilla,2422583,NA - ENG - CR - Tier 1,0,0,01/01/2020,00:00:44,42,4,1259,0,1305,0,4,N,0,Y
3,123178879722,123178879722,18959143,Phone Call,18006802345,+16365771554,4069210,Default Inbound - NO AGENT,2522967,Admin,0,NaN,0,NaN,0,-1,01/01/2020,00:00:59,5,0,0,0,5,0,0,N,0,N
4,123178880296,123178880296,18959143,Phone Call,18006802345,+12055332250,4149348,NA - ENG - Care - IB,2523910,Americas - Eng Care,15325183,Angel Rodriguez,2421928,NA - ENG - LH - Tier 1,0,0,01/01/2020,00:01:44,26,4,5163,0,5193,0,4,N,0,Y


# Class testing with full date range 

In [8]:
call_detail = InContactReport(start_date='2019-01-01', end_date='2020-02-01')

In [9]:
call_detail.date_batches

[('2019-01-01T00:00:00', '2019-01-31T00:00:00'),
 ('2019-02-01T00:00:00', '2019-02-28T00:00:00'),
 ('2019-03-01T00:00:00', '2019-03-31T00:00:00'),
 ('2019-04-01T00:00:00', '2019-04-30T00:00:00'),
 ('2019-05-01T00:00:00', '2019-05-31T00:00:00'),
 ('2019-06-01T00:00:00', '2019-06-30T00:00:00'),
 ('2019-07-01T00:00:00', '2019-07-31T00:00:00'),
 ('2019-08-01T00:00:00', '2019-08-31T00:00:00'),
 ('2019-09-01T00:00:00', '2019-09-30T00:00:00'),
 ('2019-10-01T00:00:00', '2019-10-31T00:00:00'),
 ('2019-11-01T00:00:00', '2019-11-30T00:00:00'),
 ('2019-12-01T00:00:00', '2019-12-31T00:00:00'),
 ('2020-01-01T00:00:00', '2020-01-31T00:00:00')]

In [10]:
report_id = '524'

call_detail.run_report(report_id)

Starting Report 524 for date batch 2019-01-01T00:00:00 -> 2019-01-31T00:00:00
Batch 2019-01-01T00:00:00 -> 2019-01-31T00:00:00 finished
Starting Report 524 for date batch 2019-02-01T00:00:00 -> 2019-02-28T00:00:00
Batch 2019-02-01T00:00:00 -> 2019-02-28T00:00:00 finished
Starting Report 524 for date batch 2019-03-01T00:00:00 -> 2019-03-31T00:00:00
Batch 2019-03-01T00:00:00 -> 2019-03-31T00:00:00 finished
Starting Report 524 for date batch 2019-04-01T00:00:00 -> 2019-04-30T00:00:00
Batch 2019-04-01T00:00:00 -> 2019-04-30T00:00:00 finished
Starting Report 524 for date batch 2019-05-01T00:00:00 -> 2019-05-31T00:00:00
Batch 2019-05-01T00:00:00 -> 2019-05-31T00:00:00 finished
Starting Report 524 for date batch 2019-06-01T00:00:00 -> 2019-06-30T00:00:00
Batch 2019-06-01T00:00:00 -> 2019-06-30T00:00:00 finished
Starting Report 524 for date batch 2019-07-01T00:00:00 -> 2019-07-31T00:00:00
Batch 2019-07-01T00:00:00 -> 2019-07-31T00:00:00 finished
Starting Report 524 for date batch 2019-08-01T00

In [12]:
full_call_detail_df = pd.concat(call_detail.report_df_batches, ignore_index=True, sort=False)

In [13]:
full_call_detail_df.head()

,contact_id,master_contact_id,Contact_Code,media_name,contact_name,ANI_DIALNUM,skill_no,skill_name,campaign_no,campaign_name,agent_no,agent_name,team_no,team_name,disposition_code,SLA,start_date,start_time,PreQueue,InQueue,Agent_Working_Time,PostQueue,Total_Time,Abandon_Time,Routing_Time,abandon,callback_time,Logged
0,36821683098,36821683098,18959116,Phone Call,+31357994014,+33666227113,4149885,EU - FRE - Tier 1,2523907,EMEA - French,7091398,Pierre Bertoldi,2421945,EU - French Tier 1,0,0,01/02/2019,16:32:15,25.0,1.0,1621.0,0.0,1647.0,0.0,1,N,0,Y
1,36821683113,36821683113,18959117,Phone Call,+31357994015,+4935952499351,4149879,EU - GER - Tier 1,2523902,EMEA - German,7068125,Pascal Vogt,2421938,EU - German Tier 1,0,0,01/02/2019,16:32:15,23.0,8.0,271.0,0.0,302.0,0.0,8,N,0,Y
2,36821683041,36821683041,18961199,Phone Call,+528000835513,+18053295832,4069210,Default Inbound - NO AGENT,2522967,Admin,0,NaN,0,NaN,0,-1,01/02/2019,16:32:13,3.0,0.0,0.0,0.0,3.0,0.0,0,N,0,N
3,36821682373,36821682373,7,Phone Call,7724921559,7724921559,4160457,NA - ENG - Sales Help - OB,2524213,Americas - Sales Help,6972665,Megan Greer,2422271,NA - ENG - SB - Sales,0,-1,01/02/2019,16:31:57,0.0,0.0,59.0,0.0,59.0,0.0,0,N,0,N
4,36821682226,36821682226,18959187,Phone Call,+31357994025,+441246459311,4149877,EU - ENG - Tier 1,2523901,EMEA - English,6942245,Damian Martin,2421936,EU - English Tier 1,0,0,01/02/2019,16:31:54,44.0,163.0,831.0,0.0,1038.0,0.0,11,N,0,Y


In [14]:
full_call_detail_df.shape

(1385668, 28)

In [15]:
full_call_detail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1385668 entries, 0 to 1385667
Data columns (total 28 columns):
contact_id            1385668 non-null int64
master_contact_id     1385668 non-null int64
Contact_Code          1385668 non-null int64
media_name            1385668 non-null object
contact_name          1385604 non-null object
ANI_DIALNUM           1385604 non-null object
skill_no              1385668 non-null int64
skill_name            1385668 non-null object
campaign_no           1385668 non-null int64
campaign_name         1385668 non-null object
agent_no              1385668 non-null int64
agent_name            817680 non-null object
team_no               1385668 non-null int64
team_name             817680 non-null object
disposition_code      1385668 non-null int64
SLA                   1385668 non-null int64
start_date            1385668 non-null object
start_time            1385668 non-null object
PreQueue              1385169 non-null float64
InQueue               1

In [18]:
print('2019-01-01T00:00:00'[0:10])

2019-01-01
